In [2]:
import pandas as pd

# Load the data
df = pd.read_csv('Nat_Gas.csv')

# Ensure dates are datetime objects for easy comparison
df['Dates'] = pd.to_datetime(df['Dates'])

# The prices should automatically parse, but you can force them to float just in case
df['Prices'] = df['Prices'].astype(float)

# Set the date as the index so we can easily look up prices later
df.set_index('Dates', inplace=True)

/var/folders/2h/yq66pk6517xbj3ltp9rn__4h0000gn/T/ipykernel_23693/1561411654.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [3]:
def price_contract(in_dates, out_dates, price_data, rate, max_vol, storage_cost_total):
    """
    in_dates: List of dates to inject gas
    out_dates: List of dates to withdraw gas
    price_data: The dataframe containing historical prices
    rate: The volume of gas injected/withdrawn per action
    max_vol: Maximum volume the storage can hold
    storage_cost_total: A fixed cost to subtract (or calculated based on time)
    """
    volume = 0
    cash_flows = 0

    # Combine all dates and sort them to process chronologically
    # We tag them to know if it's an injection or withdrawal
    all_dates = [(d, 'inject') for d in in_dates] + \
                [(d, 'withdraw') for d in out_dates]
    
    # Sort by date
    all_dates.sort()

    for date, action in all_dates:
        # Look up the price on this date
        if date in price_data.index:
            price = price_data.loc[date, 'Prices']
            
            if action == 'inject':
                # Check if we have room to store
                if volume + rate <= max_vol:
                    volume += rate
                    # Buy gas: Cost is negative cash flow
                    cash_flows -= price * rate
                    print(f"Injected on {date} at {price}, Vol: {volume}")
                else:
                    print(f"Cannot inject on {date}, storage full!")
            
            elif action == 'withdraw':
                # Check if we have gas to sell
                if volume - rate >= 0:
                    volume -= rate
                    # Sell gas: Revenue is positive cash flow
                    cash_flows += price * rate
                    print(f"Withdrawn on {date} at {price}, Vol: {volume}")
                else:
                    print(f"Cannot withdraw on {date}, storage empty!")
        else:
            print(f"Price data missing for {date}")

    # Subtract storage costs
    final_value = cash_flows - storage_cost_total
    return final_value

In [4]:
# Example Usage
injection_dates = [pd.Timestamp('2020-10-31')]
withdrawal_dates = [pd.Timestamp('2021-12-31')]
rate = 100000  # volume unit
max_vol = 500000
storage_cost = 10000

result = price_contract(injection_dates, withdrawal_dates, df, rate, max_vol, storage_cost)
print(f"Contract Value: ${result}")

Injected on 2020-10-31 00:00:00 at 10.1, Vol: 100000
Withdrawn on 2021-12-31 00:00:00 at 11.4, Vol: 0
Contract Value: $120000.0
